# **Config and Library**

In [1]:
# For Colab. Need to run this after run time reset
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/transformer_mastery')
# path = '/content/drive/MyDrive/Colab Notebooks/transformer_mastery/metadata/'

In [2]:
# %pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00


In [1]:
from transformer import TransformerModel
from pickle import load
from tensorflow import Module
from keras_preprocessing.sequence import pad_sequences
from tensorflow import convert_to_tensor, int64, TensorArray, argmax, newaxis, transpose
from time import time
from nltk.translate.bleu_score import corpus_bleu

# **Load data set**

In [12]:
filename = "./data/german-english-test.txt"
with open(filename, 'rt') as file:
  test_data = file.read()
type(test_data)

str

In [54]:
test_data2 = test_data.split('\n')
test_data3 = [text.split('\t') for text in test_data2]
test_data3 = test_data3[:-1]
for i in range(5): print(test_data3[i])

['tom erbricht sich', 'tom is puking']
['welche gehort uns', 'which is ours']
['ich bin arzt', 'im a medic']
['scher dich fort', 'get away']
['er legte auf', 'he hung up']


# **Load model**

In [20]:
h = 8
d_k = 64
d_v = 64
d_model = 512
d_ff = 2048
n = 6

#9 7 3157 2009
enc_seq_length = 9
dec_seq_length = 7
enc_vocab_size = 3157
dec_vocab_size = 2009

In [21]:
inferencing_model = TransformerModel(enc_vocab_size, dec_vocab_size, enc_seq_length, dec_seq_length, h, d_k, d_v, d_model, d_ff, n, 0)

In [49]:
class Translate(Module):
    def __init__(self, inference_model, **kwargs):
        super(Translate, self).__init__(**kwargs)
        self.transformer = inference_model

    def load_tokenizer(self, name):
        with open(name, 'rb') as handle:
            return load(handle)

    def __call__(self, sentence):
        sentence[0] = "<START> " + sentence[0] + " <EOS>"

        enc_tokenizer = self.load_tokenizer('./metadata/enc_tokenizer.pkl')
        dec_tokenizer = self.load_tokenizer('./metadata/dec_tokenizer.pkl')

        encoder_input = enc_tokenizer.texts_to_sequences(sentence)
        encoder_input = pad_sequences(encoder_input, maxlen=enc_seq_length, padding='post')
        encoder_input = convert_to_tensor(encoder_input, dtype=int64)

        output_start = dec_tokenizer.texts_to_sequences(["<START>"])
        output_start = convert_to_tensor(output_start[0], dtype=int64)

        output_end = dec_tokenizer.texts_to_sequences(["<EOS>"])
        output_end = convert_to_tensor(output_end[0], dtype=int64)

        decoder_output = TensorArray(dtype=int64, size=0, dynamic_size=True)
        decoder_output = decoder_output.write(0, output_start)

        for i in range(dec_seq_length):
            prediction = self.transformer(encoder_input, transpose(decoder_output.stack()), training=False)
            prediction = prediction[:, -1, :] # pick the last prediction
            predicted_id = argmax(prediction, axis=-1)
            decoder_output = decoder_output.write(i+1, predicted_id)

            if predicted_id == output_end:
                break

        output = transpose(decoder_output.stack())[0]
        output = output.numpy()
        # print(f"output = {output}")

        output_str = []
        for i in range(output.shape[0]):
            key = output[i]
            if key == 0:
              output_str.append(dec_tokenizer.index_word[2])  # to fix the attend to 0 problem
            else:
              output_str.append(dec_tokenizer.index_word[key])


        output_str = output_str[1:-1]
        return ' '.join(output_str)

inferencing_model.load_weights('./metadata/weights/wghts20.ckpt')
translator = Translate(inferencing_model)

# **Inference**

In [50]:
sentence = ["er legte auf"]
print('Translated result is = ', translator(sentence))

Translated result is =  i let a smoke


# **BLEU Score**

In [ ]:
actual = []
predicted = []
time0 = time()
length = 0
for i, source in enumerate(test_data3):
    raw_src = source[0]
    raw_target = source[1]
    # if i == 400: break
    #
    translation = translator([raw_src])
    if i < 3:
        print(f"src = {raw_src}")
        print(f"target = {raw_target}")
        print(f"predict = {translation}")
        print("\n")
    # #
    actual.append([raw_target.split()])
    predicted.append(translation.split())

    length += 1
    if length % 200 ==0:
        print(f"length = {length}, run time = {time()-time0}")


src = tom erbricht sich
target = tom is puking
predict = i can swim


src = welche gehort uns
target = which is ours
predict = i prefer why


src = ich bin arzt
target = im a medic
predict = im a doctor




In [53]:
print(f'Predict time = {time()-time0}')
print('BLEU-1 %f' % corpus_bleu(actual, predicted, weights=(1.0, 0.0, 0.0, 0.0)))
print('BLEU-2 %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0.0, 0.0)))
print('BLEU-3 %f' % corpus_bleu(actual, predicted, weights=(0.33, 0.33, 0.33, 0.0)))
print('BLEU-4 %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
print(f'BLEU time = {time()-time0}')

Predict time = 2123.378015756607
BLEU-1 0.263259
BLEU-2 0.162598
BLEU-3 0.105114
BLEU-4 0.054208
BLEU time = 2123.4946296215057
